In [7]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [9]:
unsw_trset, unsw_validation, y_train, y_validation = joblib.load("unsw_trv_unprocessed.pkl")
X_test, y_test = joblib.load("unsw_test_unprocessed.pkl")

X_train_val = np.concatenate([unsw_trset, unsw_validation], axis=0)
y_train_val = np.concatenate([y_train, y_validation], axis=0)
print("Training set:" + str(X_train_val.shape))

Training set:(175318, 41)


In [ ]:
input_dim = 41         
encoding_dim = 16       # dimenzija za bottleneck sloj

input_layer = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(input_layer)
bottleneck = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(bottleneck)
output_layer = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')

autoencoder.fit(
    X_train_val, X_train_val,
    epochs=25,
    batch_size=256,
    shuffle=True,
    validation_split=0.1,
    verbose=1
)

encoder = Model(inputs=input_layer, outputs=bottleneck)

X_train_reduced = encoder.predict(X_train_val)
X_test_reduced = encoder.predict(X_test)

print("Dimenzije nakon redukcije:", X_train_reduced.shape, X_test_reduced.shape)

Epoch 1/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.2615 - val_loss: 0.0369
Epoch 2/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0314 - val_loss: 0.0199
Epoch 3/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0173 - val_loss: 0.0142
Epoch 4/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0131 - val_loss: 0.0121
Epoch 5/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0113 - val_loss: 0.0109
Epoch 6/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0102 - val_loss: 0.0100
Epoch 7/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0093 - val_loss: 0.0091
Epoch 8/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0085 - val_loss: 0.0086
Epoch 9/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0080 - val_loss: 0.0081
Epoch 10/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 11/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0074 - val_loss: 0.0074
Epoch 12/25
617/617 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [11]:
joblib.dump((X_train_reduced, y_train_val), "unsw_train_reduced_autoenc.pkl")
joblib.dump((X_test_reduced, y_test), "unsw_test_reduced_autoenc.pkl")

['unsw_test_reduced_autoenc.pkl']